Part1. Train VGG16 with quantization-aware training (15%)

 - Train for 4-bit input activation and 4-bit weight to achieve >90% accuracy. 

 - But, this time, reduce a certain convolution layer's input channel numbers to be 8 and output channel numbers to be 8. (v)

 - Also, remove the batch normalization layer after the squeezed convolution. (v)

  e.g., replace "conv -> relu -> batchnorm" with "conv -> relu"

 - This layer will be mapped on your 8x8 2D systolic array. Thus, reducing to 8 channels helps your layer's mapping in an array nicely without tiling.

 - This time, compute your "psum_recovered" such as HW5 including ReLU and compare with your prehooked input for the next layer (instead of your computed psum_ref).

 - [hint] It is recommended not to reduce the input channel of Conv layer at too early layer position because the early layer's feature map size (nij) is large incurring long verification cycles.

   (recommended location: around 27-th layer, e.g., features[27] for VGGNet)

 - Measure of success: accuracy >90%  with 8 input/output channels + error < 10^-3 for psum_recorvered for VGGNet.

In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn     

import torchvision
import torchvision.transforms as transforms

from models import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [100, 200,300]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
lr = 1e-2
weight_decay = 1e-3
epochs = 100
best_prec = 0
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
# weight decay: for regularization to prevent overfitting
     

if not os.path.exists('result'):
    os.makedirs('result')
    
fdir = 'result/'+str(model_name)+str('_1206_2bit')

if not os.path.exists(fdir):
    os.makedirs(fdir)
        
#PATH = "result/VGG16_quant1129_5/model_best.pth.tar"
#checkpoint = torch.load(PATH)
#model.load_state_dict(checkpoint['state_dict'])
#device = torch.device("cuda") 
        
        
        
for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 1.751 (1.751)	Data 0.188 (0.188)	Loss 2.5184 (2.5184)	Prec 10.938% (10.938%)
Epoch: [0][100/391]	Time 0.043 (0.063)	Data 0.002 (0.004)	Loss 1.8848 (2.0341)	Prec 34.375% (25.820%)
Epoch: [0][200/391]	Time 0.045 (0.054)	Data 0.001 (0.003)	Loss 1.7193 (1.8549)	Prec 36.719% (31.775%)
Epoch: [0][300/391]	Time 0.045 (0.051)	Data 0.001 (0.002)	Loss 1.4214 (1.7307)	Prec 46.094% (36.483%)
Validation starts
Test: [0/79]	Time 0.242 (0.242)	Loss 1.4728 (1.4728)	Prec 51.562% (51.562%)
 * Prec 47.220% 
best acc: 47.220000
Epoch: [1][0/391]	Time 0.288 (0.288)	Data 0.243 (0.243)	Loss 1.4557 (1.4557)	Prec 44.531% (44.531%)
Epoch: [1][100/391]	Time 0.045 (0.047)	Data 0.001 (0.004)	Loss 1.2104 (1.2725)	Prec 55.469% (54.332%)
Epoch: [1][200/391]	Time 0.044 (0.046)	Data 0.005 (0.003)	Loss 1.1576 (1.2265)	Prec 61.719% (56.095%)
Epoch: [1][300/391]	Time 0.046 (0.046)	Data 0.002 (0.002)	Loss 1.0519 (1.1833)	Prec 60.938% (57.714%)
Validation starts
Test: [0/79]	Time 0.265 (0.265)	Loss 1.

In [2]:
PATH = "result/VGG16_quant1130_2_newmodel/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9127/10000 (91%)



In [3]:
## Send an image and use prehook to grab the inputs of all the QuantConv2d layers

class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
device = torch.device("cuda" if use_gpu else "cpu") 
counter =0
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        counter += 1
        print(layer, counter)
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

prehooked
QuantConv2d(
  3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 1
prehooked
QuantConv2d(
  64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 2
prehooked
QuantConv2d(
  64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 3
prehooked
QuantConv2d(
  128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 4
prehooked
QuantConv2d(
  128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 5
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 6
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 7
prehooked
QuantConv2d(
  2

In [10]:
w_bit = 4
quantConv2d = model.features[27]
print(quantConv2d)
#weight_q = model.layer1[0].conv2.weight_q # quantized value is stored during the training
weight_q = quantConv2d.weight_q
print(weight_q)

weight_alpha = quantConv2d.weight_quant.wgt_alpha
print(weight_alpha)
w_delta = weight_alpha / (2 ** (w_bit - 1) - 1)   # delta can be calculated by using alpha and w_bit
weight_int =  weight_q / w_delta# w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

QuantConv2d(
  8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
Parameter containing:
tensor([[[[-0.6771,  0.3386,  0.3386],
          [ 0.0000,  1.6928,  1.0157],
          [ 1.3543,  2.0314,  1.3543]],

         [[-0.3386, -0.6771, -0.6771],
          [-1.0157, -0.3386,  0.0000],
          [-0.6771,  0.6771,  0.3386]],

         [[-1.6928, -1.0157, -0.0000],
          [ 1.6928,  1.0157,  0.6771],
          [ 1.0157, -0.3386, -0.3386]],

         [[ 0.6771, -0.3386,  0.6771],
          [-1.3543, -0.0000, -0.0000],
          [-0.6771,  0.0000, -0.0000]],

         [[-0.6771, -1.3543, -1.0157],
          [-0.6771, -0.6771, -0.6771],
          [-0.0000, -1.0157, -0.3386]],

         [[-1.6928,  0.6771,  1.3543],
          [-2.3700,  0.6771,  1.6928],
          [-0.3386, -0.0000,  0.0000]],

         [[-0.6771, -0.6771,  0.6771],
          [-1.0157, -0.6771, -0.6771],
          [-2.3700, -0.6771, -0.3386]],

         [[ 0.0000,  

In [11]:
##### Find "weight_int" for features[3] ####
w_bit = 4
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha
w_delta = w_alpha /(2**(w_bit-1)-1)

weight_int = weight_q / w_delta
print(weight_int)

tensor([[[[-2.0000,  1.0000,  1.0000],
          [ 0.0000,  5.0000,  3.0000],
          [ 4.0000,  6.0000,  4.0000]],

         [[-1.0000, -2.0000, -2.0000],
          [-3.0000, -1.0000,  0.0000],
          [-2.0000,  2.0000,  1.0000]],

         [[-5.0000, -3.0000, -0.0000],
          [ 5.0000,  3.0000,  2.0000],
          [ 3.0000, -1.0000, -1.0000]],

         [[ 2.0000, -1.0000,  2.0000],
          [-4.0000, -0.0000, -0.0000],
          [-2.0000,  0.0000, -0.0000]],

         [[-2.0000, -4.0000, -3.0000],
          [-2.0000, -2.0000, -2.0000],
          [-0.0000, -3.0000, -1.0000]],

         [[-5.0000,  2.0000,  4.0000],
          [-7.0000,  2.0000,  5.0000],
          [-1.0000, -0.0000,  0.0000]],

         [[-2.0000, -2.0000,  2.0000],
          [-3.0000, -2.0000, -2.0000],
          [-7.0000, -2.0000, -1.0000]],

         [[ 0.0000,  3.0000,  4.0000],
          [-2.0000,  4.0000,  6.0000],
          [-6.0000, -4.0000,  1.0000]]],


        [[[-1.0000,  5.0000,  4.0000],
       

In [12]:
act = save_output.outputs[8][0] # input of 27th quantconv
act_alpha  = model.features[27].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)
act_q = act_quant_fn(act, act_alpha)
act_delta = act_alpha / (2**act_bit - 1)
act_int = act_q / act_delta
print(act_int)

tensor([[[[ 0.0000,  0.0000,  0.0000,  2.0000],
          [ 0.0000,  1.0000,  2.0000,  2.0000],
          [ 0.0000,  0.0000,  1.0000,  2.0000],
          [ 1.0000,  1.0000,  1.0000,  2.0000]],

         [[ 6.0000,  4.0000,  1.0000,  2.0000],
          [ 2.0000,  0.0000,  2.0000,  3.0000],
          [ 1.0000,  0.0000,  0.0000,  1.0000],
          [ 2.0000,  1.0000,  1.0000,  2.0000]],

         [[ 2.0000,  2.0000,  1.0000,  0.0000],
          [ 3.0000,  3.0000,  2.0000,  1.0000],
          [ 2.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         ...,

         [[ 0.0000,  0.0000,  0.0000,  1.0000],
          [ 1.0000,  0.0000,  0.0000,  1.0000],
          [ 1.0000,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  2.0000,  3.0000,  2.0000],
          [ 0.0000,  1.0000,  3.0000,  1.0000],
          [ 0.0000,  2.0000,  2.0000,  1.0000],
          [ 1.0000,  2.0000,  1.0000,  1.0000]],

         [[ 0.0

In [13]:
out_ref = save_output.outputs[9][0]

In [14]:
conv_int = torch.nn.Conv2d(8,8, kernel_size = 3, padding = 1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
out_int = conv_int(act_int)
relu = model.features[28]
out_recovered = out_int * act_delta * w_delta
out_recovered = relu(out_recovered)


In [15]:
difference = abs(out_ref - out_recovered)
print(difference.mean())

tensor(5.2647e-07, device='cuda:0', grad_fn=<MeanBackward0>)
